# 新しいseedパラメータを使用して補完出力を再現可能にする方法

**要約**: 開発者は、Chat Completionリクエストで`seed`パラメータを指定することで、（ほぼ）一貫した出力を受け取ることができるようになりました。これらの変更を追跡できるよう、`system_fingerprint`フィールドを公開しています。この値が異なる場合、システムに加えた変更により異なる出力が表示される可能性があります。この機能はベータ版であり、現在は`gpt-4-1106-preview`と`gpt-3.5-turbo-1106`でのみサポートされていることにご注意ください。

### 背景

APIを使用する際の再現性は、ユーザーコミュニティから常に大きな要望がありました。例えば、再現可能な数値結果を得る機能が提供されることで、ユーザーは数値の変化に敏感な多くのユースケースを実現できるようになります。

#### 一貫した出力のためのモデルレベル機能

Chat CompletionsおよびCompletions APIは、デフォルトでは非決定論的です（つまり、リクエストごとにモデル出力が異なる可能性があります）が、いくつかのモデルレベル制御を使用して決定論的出力をある程度制御できるようになりました。

これにより一貫した補完が可能になり、API上に構築されたあらゆるもののモデル動作を完全に制御できるようになります。また、結果の再現やテストに非常に有用で、何が得られるかを正確に知ることで安心感を得ることができます。

#### 一貫した出力の実装

API呼び出し間で_ほぼ_決定論的な出力を受け取るには：

- `seed`パラメータを任意の整数に設定しますが、リクエスト間で同じ値を使用してください。例：`12345`
- その他のすべてのパラメータ（プロンプト、temperature、top_p等）をリクエスト間で同じ値に設定してください
- レスポンスで`system_fingerprint`フィールドを確認してください。システムフィンガープリントは、OpenAIサーバーが補完を生成するために使用するモデルの重み、インフラストラクチャ、その他の設定オプションの現在の組み合わせの識別子です。リクエストパラメータを変更したり、OpenAIがモデルを提供するインフラストラクチャの数値設定を更新したりするたびに変更されます（これは年に数回発生する可能性があります）。

`seed`、リクエストパラメータ、および`system_fingerprint`がすべてリクエスト間で一致する場合、モデル出力はほぼ同一になります。モデル固有の非決定性により、リクエストパラメータと`system_fingerprint`が一致していても、レスポンスが異なる小さな可能性があります。

### 一貫した出力のためのモデルレベル制御 - `seed` と `system_fingerprint`

##### `seed`

指定された場合、システムは決定論的にサンプリングするよう最善の努力を行い、同じseedとパラメータを持つ繰り返しリクエストは同じ結果を返すようになります。決定論性は保証されておらず、バックエンドの変更を監視するために `system_fingerprint` レスポンスパラメータを参照する必要があります。

##### `system_fingerprint`

このフィンガープリントは、モデルが実行されるバックエンド設定を表します。seedリクエストパラメータと組み合わせて使用することで、決定論性に影響を与える可能性のあるバックエンドの変更がいつ行われたかを理解できます。これは、ユーザーが「ほぼ常に同じ結果」を期待すべきかどうかの指標です。

## 例：固定シードを使用した短い抜粋の生成

この例では、固定シードを使用して短い抜粋を生成する方法を説明します。これは、テスト、デバッグ、または一貫した出力を必要とするアプリケーションにおいて、一貫した結果を生成する必要があるシナリオで特に有用です。

### Python SDK

> **注意**
> SDKの最新バージョン（執筆時点では`1.3.3`）に切り替えてください。

In [ ]:
!pip install --upgrade openai # Switch to the latest version of OpenAI (1.3.3 at time of writing)

In [12]:
import openai
import asyncio
from IPython.display import display, HTML

from utils.embeddings_utils import (
    get_embedding,
    distances_from_embeddings
)

GPT_MODEL = "gpt-3.5-turbo-1106"

In [13]:
async def get_chat_response(
    system_message: str, user_request: str, seed: int = None, temperature: float = 0.7
):
    try:
        messages = [
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_request},
        ]

        response = openai.chat.completions.create(
            model=GPT_MODEL,
            messages=messages,
            seed=seed,
            max_tokens=200,
            temperature=temperature,
        )

        response_content = response.choices[0].message.content
        system_fingerprint = response.system_fingerprint
        prompt_tokens = response.usage.prompt_tokens
        completion_tokens = response.usage.total_tokens - response.usage.prompt_tokens

        table = f"""
        <table>
        <tr><th>Response</th><td>{response_content}</td></tr>
        <tr><th>System Fingerprint</th><td>{system_fingerprint}</td></tr>
        <tr><th>Number of prompt tokens</th><td>{prompt_tokens}</td></tr>
        <tr><th>Number of completion tokens</th><td>{completion_tokens}</td></tr>
        </table>
        """
        display(HTML(table))

        return response_content
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

def calculate_average_distance(responses):
    """
    This function calculates the average distance between the embeddings of the responses.
    The distance between embeddings is a measure of how similar the responses are.
    """
    # Calculate embeddings for each response
    response_embeddings = [get_embedding(response) for response in responses]

    # Compute distances between the first response and the rest
    distances = distances_from_embeddings(response_embeddings[0], response_embeddings[1:])

    # Calculate the average distance
    average_distance = sum(distances) / len(distances)

    # Return the average distance
    return average_distance

まず、`seed`パラメータを使わずに「火星への旅」についての短い抜粋をいくつかの異なるバージョンで生成してみましょう。これがデフォルトの動作です：

In [14]:
topic = "a journey to Mars"
system_message = "You are a helpful assistant."
user_request = f"Generate a short excerpt of news about {topic}."

responses = []


async def get_response(i):
    print(f'Output {i + 1}\n{"-" * 10}')
    response = await get_chat_response(
        system_message=system_message, user_request=user_request
    )
    return response


responses = await asyncio.gather(*[get_response(i) for i in range(5)])
average_distance = calculate_average_distance(responses)
print(f"The average similarity between responses is: {average_distance}")

Output 1
----------


Response,"""NASA's Mars mission reaches critical stage as spacecraft successfully enters orbit around the red planet. The historic journey, which began over a year ago, has captured the world's attention as scientists and astronauts prepare to land on Mars for the first time. The mission is expected to provide valuable insights into the planet's geology, atmosphere, and potential for sustaining human life in the future."""
System Fingerprint,fp_772e8125bb
Number of prompt tokens,29
Number of completion tokens,76


Output 2
----------


Response,"""NASA's Perseverance rover successfully landed on Mars, marking a major milestone in the mission to explore the red planet. The rover is equipped with advanced scientific instruments to search for signs of ancient microbial life and collect samples of rock and soil for future return to Earth. This historic achievement paves the way for further exploration and potential human missions to Mars in the near future."""
System Fingerprint,fp_772e8125bb
Number of prompt tokens,29
Number of completion tokens,76


Output 3
----------


...

Output 4
----------


Response,"""NASA's latest Mars mission exceeds expectations as the Perseverance rover uncovers tantalizing clues about the Red Planet's past. Scientists are thrilled by the discovery of ancient riverbeds and sedimentary rocks, raising hopes of finding signs of past life on Mars. With this exciting progress, the dream of sending humans to Mars feels closer than ever before."""
System Fingerprint,fp_772e8125bb
Number of prompt tokens,29
Number of completion tokens,72


Output 5
----------


Response,"""NASA's Perseverance Rover Successfully Lands on Mars, Begins Exploration Mission In a historic moment for space exploration, NASA's Perseverance rover has successfully landed on the surface of Mars. After a seven-month journey, the rover touched down in the Jezero Crater, a location scientists believe may have once held a lake and could potentially contain signs of ancient microbial life. The rover's primary mission is to search for evidence of past life on Mars and collect rock and soil samples for future return to Earth. Equipped with advanced scientific instruments, including cameras, spectrometers, and a drill, Perseverance will begin its exploration of the Martian surface, providing valuable data and insights into the planet's geology and potential habitability. This successful landing marks a significant milestone in humanity's quest to understand the red planet and paves the way for future manned missions to Mars. NASA's Perseverance rover is poised to unravel the mysteries of Mars and unlock new possibilities"
System Fingerprint,fp_772e8125bb
Number of prompt tokens,29
Number of completion tokens,200


The average similarity between responses is: 0.1136714512418833


それでは、同じコードを定数`seed`を123、`temperature`を0に設定して実行し、レスポンスと`system_fingerprint`を比較してみましょう。

In [15]:
SEED = 123
responses = []


async def get_response(i):
    print(f'Output {i + 1}\n{"-" * 10}')
    response = await get_chat_response(
        system_message=system_message,
        seed=SEED,
        temperature=0,
        user_request=user_request,
    )
    return response


responses = await asyncio.gather(*[get_response(i) for i in range(5)])

average_distance = calculate_average_distance(responses)
print(f"The average distance between responses is: {average_distance}")

Output 1
----------


Response,"""NASA's Perseverance Rover Successfully Lands on Mars In a historic achievement, NASA's Perseverance rover has successfully landed on the surface of Mars, marking a major milestone in the exploration of the red planet. The rover, which traveled over 293 million miles from Earth, is equipped with state-of-the-art instruments designed to search for signs of ancient microbial life and collect rock and soil samples for future return to Earth. This mission represents a significant step forward in our understanding of Mars and the potential for human exploration of the planet in the future."""
System Fingerprint,fp_772e8125bb
Number of prompt tokens,29
Number of completion tokens,113


Output 2
----------


Response,"""NASA's Perseverance rover successfully lands on Mars, marking a historic milestone in space exploration. The rover is equipped with advanced scientific instruments to search for signs of ancient microbial life and collect samples for future return to Earth. This mission paves the way for future human exploration of the red planet, as scientists and engineers continue to push the boundaries of space travel and expand our understanding of the universe."""
System Fingerprint,fp_772e8125bb
Number of prompt tokens,29
Number of completion tokens,81


Output 3
----------


...

Output 4
----------


Response,"""NASA's Perseverance rover successfully lands on Mars, marking a historic milestone in space exploration. The rover is equipped with advanced scientific instruments to search for signs of ancient microbial life and collect samples for future return to Earth. This mission paves the way for future human exploration of the red planet, as scientists and engineers continue to push the boundaries of space travel and expand our understanding of the universe."""
System Fingerprint,fp_772e8125bb
Number of prompt tokens,29
Number of completion tokens,81


Output 5
----------


Response,"""NASA's Perseverance rover successfully lands on Mars, marking a historic milestone in space exploration. The rover is equipped with advanced scientific instruments to search for signs of ancient microbial life and collect samples for future return to Earth. This mission paves the way for future human exploration of the red planet, as scientists and engineers continue to push the boundaries of space travel."""
System Fingerprint,fp_772e8125bb
Number of prompt tokens,29
Number of completion tokens,74


The average distance between responses is: 0.0449054397632461


観察できるように、`seed`パラメータを使用することで、はるかに一貫性のある結果を生成することができます。

## 結論

固定された整数の`seed`を使用してモデルから一貫した出力を生成する方法を実証しました。これは再現性が重要なシナリオで特に有用です。ただし、`seed`は一貫性を保証しますが、出力の品質を保証するものではないことに注意することが重要です。再現可能な出力を使用したい場合は、Chat Completions呼び出し全体で`seed`を同じ整数に設定する必要があります。また、`temperature`、`max_tokens`などの他のパラメータも一致させる必要があります。再現可能な出力のさらなる拡張として、異なるプロンプトやモデルのパフォーマンスをベンチマーク/評価する際に一貫した`seed`を使用することが考えられます。これにより、各バージョンが同じ条件下で評価され、比較が公正で結果が信頼できるものになります。